# Simple Stateful Chatbot with LangGraph and ChatAmazonNova

This notebook demonstrates building a basic stateful chatbot using LangGraph and Amazon Nova.

The chatbot maintains conversation history and uses LangGraph's state management to provide contextual responses.

## Key Concepts
- **MessagesState**: Built-in state for message tracking
- **State Persistence**: Conversation history maintained across turns
- **History Management**: Automatic trimming to manage context window

## Setup

First, ensure you have the required packages installed and environment variables set.

In [ ]:
# Install required packages (uncomment if needed)
# !pip install langchain-nova langgraph langchain-core

import os

# Set environment variables (or use .env file)
# os.environ["NOVA_API_KEY"] = "your-api-key"
# os.environ["NOVA_BASE_URL"] = "https://api.nova.amazon.com/v1"

## Import Dependencies

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState

from langchain_amazon_nova import ChatAmazonNova

## Define the Chatbot Node

The chatbot node processes messages with conversation history.

In [ ]:
def call_model(state: MessagesState, llm) -> dict:
    """Call the model with conversation history."""
    messages = state["messages"]
    
    # Add system message if this is the first turn
    if len(messages) == 1:
        system_msg = SystemMessage(
            content="You are a helpful assistant. Be conversational and remember context from earlier in the conversation."
        )
        messages = [system_msg] + messages
    
    response = llm.invoke(messages)
    
    return {"messages": [response]}

## Create the LangGraph

Build the graph with a single node for the chatbot.

In [ ]:
def create_chatbot_graph(llm):
    """Create a simple stateful chatbot graph."""
    workflow = StateGraph(MessagesState)
    
    workflow.add_node("chatbot", lambda state: call_model(state, llm))
    
    workflow.set_entry_point("chatbot")
    workflow.set_finish_point("chatbot")
    
    return workflow.compile()

## Initialize the Model and Chatbot

In [ ]:
# Initialize ChatAmazonNova
llm = ChatAmazonNova(
    model="nova-pro-v1",
    temperature=0.7,
)

# Create chatbot
chatbot = create_chatbot_graph(llm)

# Initialize conversation state
conversation_state = {"messages": []}

print("Chatbot initialized! Ready to chat.")

## Chat with the Bot

Now you can have a conversation! The state persists across cells.

In [ ]:
# First turn
user_message = "Hi! My name is Alice."

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

In [ ]:
# Second turn - bot should remember your name
user_message = "What's my name?"

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

In [ ]:
# Third turn - continue the conversation
user_message = "Tell me a fun fact about space."

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")

## View Conversation History

In [ ]:
print(f"Conversation has {len(conversation_state['messages'])} messages:\n")

for i, msg in enumerate(conversation_state["messages"]):
    role = "User" if msg.type == "human" else "Assistant"
    content = msg.content[:100] + "..." if len(msg.content) > 100 else msg.content
    print(f"{i+1}. {role}: {content}\n")

## Clear Conversation and Start Fresh

In [ ]:
# Reset the conversation
conversation_state = {"messages": []}
print("Conversation cleared!")

## Try It Yourself

Use the cell below to continue chatting!

In [ ]:
# Your turn - modify the message below
user_message = "Your message here"

conversation_state["messages"].append(HumanMessage(content=user_message))
result = chatbot.invoke(conversation_state)
conversation_state = result

print(f"You: {user_message}")
print(f"\nAssistant: {result['messages'][-1].content}")